# Calculator Sample Client
This sample is focusing purely on building the client using [EZTZ](https://github.com/TezTech/eztz) Javascript library. Contract and all instrumentation code is inherited from [Calculator Dapp sample](calculator_dapp.ipynb), however commentary is skipped to reduce noice. To get more details on instrumentation please consult mentioned sample.

## Sandbox Environment Preparation

In [1]:
# Initialisation of variables to be used later in tutorial
export TEZOS_HOME=~/tezos-dev/tezos
export TEZOS_NODE_URL="127.0.0.1:18731"

# This variable will disable disclaimer about used tezos network. For this guide we will 
# disable it to make command outputs less cluttered. However we strongly discourage to 
# use this setting for your regular work. (source : Tezos Source code reverse engineering)
export TEZOS_CLIENT_UNSAFE_DISABLE_DISCLAIMER=yes
cd $TEZOS_HOME

./src/bin_node/tezos-sandboxed-node.sh \
  1 \
  --cors-header="*" \
  --cors-origin="*" \
  --connections 1 \
  1>tezos_sandbox_log.txt 2>&1 &

#Exporting node pid so we can terminate the sandbox node after the tutorial.
export NODE_PID=$! 
echo $NODE_PID > tezos_sandbox_pid.txt

[1] 51714


In [2]:
#It is required to run following script with the same parameter as the sandbox node was executed (in our case 1)
cd $TEZOS_HOME
eval `./src/bin_client/tezos-init-sandboxed-client.sh 1`

## Tezos address added: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx
## Tezos address added: tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN
## Tezos address added: tz1faswCTDciRzE4oJ9jn2Vm2dvjeyA9fUzU
## Tezos address added: tz1b7tUupMgCNw2cCLpKTkSD1NZzB5TkP2sv
## Tezos address added: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
## Tezos address added: tz1TGu6TN5GSez2ndXXeDX6LgUDvLzPLqgYV
## 
## The client is now properly initialized. In the rest of this shell
## session, you might now run `tezos-client` to communicate with a
## tezos node launched with `launch-sandboxed-node 1`. For instance:
## 
##   tezos-client rpc get /chains/main/blocks/head/metadata
## 
## Note: if the current protocol version, as reported by the previous
## command, is "Ps6mwMrF2ER2s51cp9yYpjDcuzQjsc2yAz8bQsRgdaRxw4Fk95H", you
## may have to activate in your "sandboxed network" the same economic
## protocol than used by the alphanet by running:
## 
##   tezos-activate-alpha
## 
## Warning: all the client data will be removed when you cl

In [3]:
tezos-activate-alpha 

tezos-client bake for bootstrap1

Injected BMJBxBKS7zLM
Injected block BMMhusR4mEF5


## EZTZ Client Library Environment
For the sample we will use [EZTZ Javascript Library](https://github.com/TezTech/eztz). You can find documentation [here](https://github.com/TezTech/eztz/wiki/Documentation#eztz.crypto). EZTZ library is able to communicate with Tezos RPC and also provides crypto functions for signing.

At the time of writing this guide, EZTZ is not exported as NPM model. To make it work we need to prepare the project manually and fetch the library directly from github.

Following steps are required when running EZTZ using nodejs. When embedding EZTZ in browser, it is required to embedd [eztz.min.js](https://github.com/TezTech/eztz/raw/master/dist/eztz.min.js) file.


In [4]:
mkdir -p ~/tezos-dev/calculator_dapp_client && cd ~/tezos-dev/calculator_dapp_client

# Fetch of dependencies for 
npm init -y
npm install --save bs58check libsodium-wrappers bip39 xhr2 bignumber.js

wget https://github.com/TezTech/eztz/raw/master/src/main.js -O eztz.js

Wrote to /Users/martinkovacik/tezos-dev/calculator_dapp_client/package.json:

{
  "name": "calculator_dapp_client",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "dependencies": {
    "bignumber.js": "^8.0.2",
    "bip39": "^2.5.0",
    "bs58check": "^2.1.2",
    "libsodium-wrappers": "^0.7.4",
    "xhr2": "^0.1.4"
  },
  "devDependencies": {},
  "description": ""
}


npm WARN calculator_dapp_client@1.0.0 No descriptionpostinstall: xhr2
npm WARN calculator_dapp_client@1.0.0 No repository field.

+ bignumber.js@8.0.2░░░░⸩ ⠏ postinstall: WARN calculator_dapp_client@1.0.0 No reposi
+ libsodium-wrappers@0.7.4
+ bip39@2.5.0
+ bs58check@2.1.2
+ xhr2@0.1.4
updated 5 packages and audited 114 packages in 6.05s
found 0 vulnerabilities

--2019-01-26 22:02:24--  https://github.com/TezTech/eztz/raw/master/src/main.js
Resolving github.com (github.com)... 140.82.118.4, 1

## Preparation of Test Account to be used by EZTZ
When interacting with tezos, it is required to have private/public key pair. This represents account, where we can move some tez and later to communicate with our smart contract.

Following code is equivalent to calling `tezos-client gen keys ...` command ([sample](calculator_dapp.ipynb#Testing-account-creation)). 

**WORKING NOTE: At the time of writing this guide, we were not able to use `tezos-client` generated keys in EZTZ, however keys generated by EZTZ work with tezos-client**.

Following call will generate private/public key, that we will later use for moving tez to this account and also to originate and call testing contract.

In [5]:
node <<EOM 2>/dev/null | tee keys.json
require('./eztz.js');

var node_address ='$TEZOS_NODE_URL';

(async() => {
    await eztz.library.sodium.ready;

    eztz.node.setProvider(node_address);

    var keys = eztz.crypto.generateKeysNoSeed()
    console.log(JSON.stringify(keys));
})();
EOM

export SECRET_KEY=unencrypted:$(jq -r ".sk" <keys.json)
export PUBLIC_KEY=$(jq -r ".pk" <keys.json)
export PUBLIC_KEY_HASH=$(jq -r ".pkh" <keys.json)

{"sk":"edskRukRLGC2N3Zsptz1NB2xstC7sG1j5xvtMZ6ugUHtgeGwAcsnkLRc2TcNq7HkiBJqaWun81DvQHQmkXDEPayRpCgGPG5KAp","pk":"edpkuFS8rHXMZxsUNxhZoM9jomkTBYtJrK1PinESSuWojQ66rToHiV","pkh":"tz1RmU5xid1hMMTubCEhv9raAttiRx7Xmprs"}


For convenience we will register given key pair with tezos-client. After registering the key we can validate, that generated public key and hash are the same as values provided by EZTZ.

In [7]:
tezos-client import secret key client_account $SECRET_KEY 
echo ----
echo Tezos-client registered keys:
tezos-client show address client_account -S
echo ----
echo EZTZ Generated keys:
jq <keys.json

Error:
  The secret_key alias client_account already exists.
    The current value is unencrypted:edskRukRLGC2N3Zsptz1NB2xstC7sG1j5xvtMZ6ugUHtgeGwAcsnkLRc2TcNq7HkiBJqaWun81DvQHQmkXDEPayRpCgGPG5KAp.
    Use --force to update
----
Tezos-client registered keys:
Hash: tz1RmU5xid1hMMTubCEhv9raAttiRx7Xmprs
Public Key: edpkuFS8rHXMZxsUNxhZoM9jomkTBYtJrK1PinESSuWojQ66rToHiV
Secret Key: unencrypted:edskRukRLGC2N3Zsptz1NB2xstC7sG1j5xvtMZ6ugUHtgeGwAcsnkLRc2TcNq7HkiBJqaWun81DvQHQmkXDEPayRpCgGPG5KAp
----
EZTZ Generated keys:
{
  "sk": "edskRukRLGC2N3Zsptz1NB2xstC7sG1j5xvtMZ6ugUHtgeGwAcsnkLRc2TcNq7HkiBJqaWun81DvQHQmkXDEPayRpCgGPG5KAp",
  "pk": "edpkuFS8rHXMZxsUNxhZoM9jomkTBYtJrK1PinESSuWojQ66rToHiV",
  "pkh": "tz1RmU5xid1hMMTubCEhv9raAttiRx7Xmprs"
}


We will move some tez to our newly created account. These will be later used to pay fees for running our smart contract.

In [9]:
tezos-client transfer 1000 from bootstrap5 to client_account --burn-cap 0.257 > output.txt &
PROCESS_PID=$!

tezos-client bake for bootstrap1
wait $PROCESS_PID
cat output.txt

[2] 51807
Injected block BLyBZ3SbmMDR
[2]+  Done                    tezos-client transfer 1000 from bootstrap5 to client_account --burn-cap 0.257 > output.txt
Node is bootstrapped, ready for injecting operations.
Estimated gas: 10100 units (will add 100 for safety)
Estimated storage: 257 bytes added (will add 20 for safety)
Operation successfully injected in the node.
Operation hash: ooHf4Ykomvjxtk4J7tPqHr9H3xxw2CkPbnggRPycpTM91eHLGHv
Waiting for the operation to be included...
Operation found in block: BLyBZ3SbmMDRBN533Y92CetwSLRL3DUFa8waW6KnQQpsYWZs4rA (pass: 3, offset: 0)
This sequence of operations was run:
  Manager signed operations:
    From: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
    Fee to the baker: ꜩ0.001275
    Expected counter: 1
    Gas limit: 10200
    Storage limit: 277 bytes
    Balance updates:
      tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv ........... -ꜩ0.001275
      fees(tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx,0) ... +ꜩ0.001275
    Transaction:
      Amount: ꜩ1000
      Fro

After the transfer we can verify balance. Following code demonstrate checking of balance using EZTZ. EZTZ library is showing balance in mutez (1000000 mutez = 1 tez).

In [11]:
echo EZTZ
echo ----
node <<EOM 
require('./eztz.js');

var pkh = '$PUBLIC_KEY_HASH';
var node_address = 'http://$TEZOS_NODE_URL';

(async() => {
    await eztz.library.sodium.ready;

    eztz.node.setProvider(node_address);

    eztz.rpc.getBalance(pkh)
        .then(balance => {
            console.log("Balance is : " + balance + " mutez");    
    }
    );
})();
EOM
echo
echo tezos-client
echo ----
## call above is equivalent to 
tezos-client get balance for client_account

EZTZ
----
Balance is : 1000000000 mutez

tezos-client
----
1000 ꜩ


## Preparation of smart contract


In [23]:
#Sample preparation
cd ~/tezos-dev/calculator_dapp_client

/bin/cat <<EOM >Calculator.liq
[%%version 0.4]
  
type storage = int

let%init storage (init_storage : int) =
  init_storage
  
let%entry add (parameter : int) storage =
  ( ([] : operation list), storage + parameter )
  
let%entry subtract (parameter : int) storage =
  ( ([] : operation list), storage - parameter )
EOM

We will deploy the contract using using bootstrap1 account.

In [24]:
BS1_HASH=$(tezos-client show address bootstrap1 --show-secret | grep Hash | cut -d" " -f2)
BS1_SECRET=$(tezos-client show address bootstrap1 --show-secret | grep Secret | cut -d":" -f3)

liquidity \
  --tezos-node $TEZOS_NODE_URL \
  --source $BS1_HASH \
  --private-key $BS1_SECRET \
  --fee 1tz \
  --amount 3tz \
  Calculator.liq \
  --deploy '0' | tee contract_deploy_output.txt
  
tezos-client bake for bootstrap1
  
export CONTRACT_HASH=$(grep "New contract" contract_deploy_output.txt | cut -d" " -f3)
export OPERATION_HASH=$(grep "New contract" contract_deploy_output.txt | cut -d" " -f7)

Main contract Calculator
Main contract Calculator
Evaluated initial storage: 0
New contract KT1U1kUzvZvroFTUZ6G4CkmQS76YB4hAnUa6 deployed in operation onhentPom7pVgMtJJHe1gLiGVJzRjbVRjF6N7uzTsJLSWRNVS3w
Injected block BLUKDWjKe6Sy


### Running of Contract Using EZTZ

Checking of contract parameters.

In [32]:
node <<EOM 
require('./eztz.js');

var client_pkh = '$PUBLIC_KEY_HASH';
var client_sk = '$SECRET_KEY';
var contract_pkh = '$CONTRACT_HASH';
var node_address = 'http://$TEZOS_NODE_URL';

(async() => {
    await eztz.library.sodium.ready;

    eztz.node.setProvider(node_address);
    
    
    //Retrieving balance of the account
    eztz.rpc.getBalance(contract_pkh)
        .then(balance => {
            console.log("Balance of contract "+contract_pkh+" is "+ balance);
        });
    
    //Retrieving storage of the account
    eztz.contract.storage(contract_pkh)
        .then(storage => {
            console.log("Storage of contract "+contract_pkh+" is : " + JSON.stringify(storage) + " mutez");  
        });
    
    //Retrieving contract object - containing all information about smart contract
    eztz.contract.load(contract_pkh)
        .then(storage => {
            console.log(storage);
        });
})();
EOM

Balance of contract KT1U1kUzvZvroFTUZ6G4CkmQS76YB4hAnUa6 is 3000000
Storage of contract KT1U1kUzvZvroFTUZ6G4CkmQS76YB4hAnUa6 is : {"int":"0"} mutez
{ manager: 'tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx',
  balance: '3000000',
  spendable: false,
  delegate: { setable: false },
  script:
   { code:
      [ { prim: 'parameter',
          args:
           [ { prim: 'or',
               args:
                [ { prim: 'int', annots: [ '%_Liq_entry_add' ] },
                  { prim: 'int', annots: [ '%_Liq_entry_subtract' ] } ],
               annots: [ ':_entries' ] } ] },
        { prim: 'storage', args: [ { prim: 'int' } ] },
        { prim: 'code',
          args:
           [ [ { prim: 'DUP' },
               { prim: 'DIP',
                 args: [ [ { prim: 'CDR', annots: [ '@storage_slash_1' ] } ] ] },
               { prim: 'CAR', annots: [ '@parameter_slash_2' ] },
               { prim: 'DUP', annots: [ '@parameter' ] },
               { prim: 'IF_LEFT',
                 args:
       

When communicating with smart contract, it's required to use Michelson. To get code required to compile method call to Michelson.


In [36]:
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --data add '1'

Main contract Calculator
(Left 1)


We can adjust resulting Michelson code and use it in our Javascript code.

**WORKING NOTE: At the time of writing EZTZ was not working when using send operation. Comparing to call with liquidity it's sending also reveal operation. It seems, that RPC call fails on reveal operation. When adjusted library manually, it was able to do the RPC call, however crashed internally**

In [56]:
node <<EOM 
require('./eztz.js');

var client_pk = '$PUBLIC_KEY';
var client_pkh = '$PUBLIC_KEY_HASH';
var client_sk = '$SECRET_KEY';
var contract_pkh = '$CONTRACT_HASH';
// var node_address = 'http://$TEZOS_NODE_URL';
 var node_address = 'http://localhost:18732';

(async() => {
    await eztz.library.sodium.ready;

    eztz.node.setProvider(node_address);
    
    //Retrieving contract object - containing all information about smart contract
    eztz.contract.send(contract_pkh, client_pkh, client_sk, 0, "(Left 1)", "0100000",100000,100000)
        .then(storage => {
            console.log(storage);
        });
})();
EOM

FORGE VALIDATION TEST START
{ branch: 'BMXRygwzgU6zCeHweFNTX3jYSr4ctckUdmMF4dY1xRsp4qqfpa6',
  contents:
   [ { kind: 'transaction',
       fee: '0100000',
       gas_limit: '100000',
       storage_limit: '100000',
       amount: '0',
       destination: 'KT1U1kUzvZvroFTUZ6G4CkmQS76YB4hAnUa6',
       parameters: { prim: 'Left', args: [ { int: '1' } ] },
       source: 'tz1RmU5xid1hMMTubCEhv9raAttiRx7Xmprs',
       counter: '2' } ] }
eec0f1e8749d425535a94ebc1fe91f300aa9d81860f84add40ab9253c0e00d130800004338dd38346932ec58318fd03801d24a9149a3fca08d0602a08d06a08d060001d523e2b5c073a12bd2f2a5cc1d5bb7abcf341bc600ff0000000405050001
eec0f1e8749d425535a94ebc1fe91f300aa9d81860f84add40ab9253c0e00d130800004338dd38346932ec58318fd03801d24a9149a3fca08d0602a08d06a08d060001d523e2b5c073a12bd2f2a5cc1d5bb7abcf341bc600ff0000000405050001
FORGE VALIDATION TEST END
(node:52329) UnhandledPromiseRejectionWarning: TypeError: Expected String
    at decodeUnsafe (/Users/martinkovacik/tezos-dev/calculator_dapp_clie

## Sandbox Clean-up

Command to terminate sandbox blockchain running in the background

In [ ]:
kill $(<$TEZOS_HOME/tezos_sandbox_pid.txt)